## Lógica Computacional: 25/26
---
## TP3 - Ex1

$Grupo$ $05$ 

*   Vasco Ferreira Leite (A108399)
*   Gustavo da Silva Faria (A108575)
*   Afonso Henrique Cerqueira Leal (A108472)
---

## Problema:

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes $a,b>0$ e devolve inteiros $r,s,t$ tais que $a*s + b*t = r$ e $r=\gcd(a,b)$.
Para além das variáveis $r,s,t$ o código requer 3 variáveis adicionais $r',s',t'$ que representam os valores de $r,s,t$ no “próximo estado”.

1.  Construa um SFOTS usando BitVector’s de tamanho $n=16$ bits que descreva o comportamento deste programa. Considere estado de erro quando $r=0$ ou alguma das variáveis atinge o “overflow”.
2.  Usando a metodologia das “Constraint Horn Clauses” (CHCs) verifique se é possível determinar um invariante que garanta que nunca se atinge um estado de erro.
3.  Verifique, usando a metodologia dos invariantes e interpolantes, se o modelo atinge um estado de erro. Para o cálculo do interpolante usar a metodologia das “Constraint Horn Clauses” (CHCs).

---
### Variáveis e Parâmetros do Modelo:

**Parâmetros de Configuração:**

- `N`: Um inteiro que define a largura dos `BitVecs` (neste caso, **16 bits**). Define o domínio finito onde ocorrem as operações do algoritmo.

- `EXT`: Um inteiro (**64 bits**) que define a largura estendida para cálculos intermédios. É usado para garantir que multiplicações e subtrações não sofram overflow ou underflow.

**Variáveis Simbólicas (Z3):**

As variáveis são representadas por `z3.BitVecs` de tamanho `N`:

- `a`, `b`: Variáveis que representam os **valores de entrada** arbitrários do algoritmo.

- `r`, `s`, `t`: Variáveis que representam o **estado atual** dos coeficientes do algoritmo Euclideano Estendido.

- `r_p`, `s_p`, `t_p`: Variáveis que representam o **próximo estado** ($r', s', t'$) após uma transição.

**Relações (Cláusulas de Horn):**

- `Inv`: Uma função não interpretada que representa o **Invariante** desconhecido que o solver tenta sintetizar. Mapeia o estado $(a, b, r, r', s, s', t, t')$ para um booleano.

- `Fail`: Uma relação booleana que se torna verdadeira se o sistema atingir um estado de erro.

---
### Célula 1: Configuração Inicial

Importa a biblioteca **Z3** e define duas constantes globais:
* `N = 16`: A largura em bits das variáveis do estado.
* `EXT = 64`: Uma largura estendida usada apenas para cálculos intermediários.

In [ ]:
from z3 import *

N = 16      
EXT = 64 

### Célula 2: `criar_variaveis`

Cria as variáveis simbólicas do tipo **BitVector** de 16 bits.
* Define pares de variáveis para o estado atual e o próximo estado:
 `r` e `r_p`, `s` e `s_p`, `t` e `t_p`.
* `a` e `b` representam as entradas iniciais do algoritmo.

In [ ]:
def criar_variaveis():
    
    a, b = BitVecs('a b', N)
    r, r_p = BitVecs('r r_p', N)
    s, s_p = BitVecs('s s_p', N)
    t, t_p = BitVecs('t t_p', N)
    
    return (a, b, r, r_p, s, s_p, t, t_p)

### Célula 3: `configurar_solver`

Inicializa o `Fixedpoint` do Z3.
* Define a *engine* como **'spacer'**, para verificar transições de estados e Cláusulas de Horn.
* Ativa a opção para imprimir a resposta no final.

In [ ]:
def configurar_solver():

    fp = Fixedpoint()
    fp.set(engine='spacer')
    fp.set('print_answer', True)
    
    return fp

### Célula 4: `registar_relacoes`

Declara as funções que o solver vai tentar resolver:
* **Inv**: O Invariante do sistema. Recebe 8 argumentos (todas as variáveis do estado) e retorna verdadeiro se o estado for válido/alcançável.
* **Fail**: Uma relação que representa um estado de erro.

In [ ]:
def registar_relacoes(fp):

    B16 = BitVecSort(N)
    
    Inv = Function('Inv', B16, B16, B16, B16, B16, B16, B16, B16, BoolSort())
    Fail = Function('Fail', BoolSort())

    fp.register_relation(Inv)
    fp.register_relation(Fail)
    
    return Inv, Fail

### Célula 5: `calc_checked_next`

Esta função realiza a operação aritmética fundamental do algoritmo ($curr - q \times next$):
1. Estende os valores de 16 bits para 64 bits para evitar perda de dados durante a multiplicação.
2. Calcula o resultado.
3. Verifica se o resultado cabe de volta em 16 bits.
4. Retorna o valor truncado e uma *flag* `is_overflow` indicando se houve violação de limites.

In [ ]:
def calc_checked_next(curr_bv, q_bv, next_bv, signed_check):

    if signed_check:
        curr_ext = SignExt(EXT - N, curr_bv)
        q_ext    = ZeroExt(EXT - N, q_bv)
        next_ext = SignExt(EXT - N, next_bv)
    else:
        curr_ext = ZeroExt(EXT - N, curr_bv)
        q_ext    = ZeroExt(EXT - N, q_bv)
        next_ext = ZeroExt(EXT - N, next_bv)

    term_ext = q_ext * next_ext
    res_ext  = curr_ext - term_ext 

    res_int = BV2Int(res_ext, signed_check)

    if signed_check:
        min_int = IntVal(- (1 << (N-1)))
        max_int = IntVal((1 << (N-1)) - 1)
    else:
        min_int = IntVal(0)
        max_int = IntVal((1 << N) - 1)

    is_overflow = Or(res_int < min_int, res_int > max_int)

    res_trunc = Extract(N-1, 0, res_ext)

    return res_trunc, is_overflow

### Célula 6: `calcular_transicao`

Aplica a lógica de passo do algoritmo euclidiano estendido:
1. Calcula o quociente $q = r / r'$.
2. Usa a função anterior (`calc_checked_next`) para calcular os novos valores de $r$, $s$ e $t$.
3. Combina as *flags* de erro numa variável `any_overflow`, que será verdadeira se qualquer uma das operações falhar.

In [ ]:
def calcular_transicao(r, r_p, s, s_p, t, t_p):

    q = UDiv(r, r_p)

    r_new, ovf_r = calc_checked_next(r, q, r_p, signed_check=False)
    s_new, ovf_s = calc_checked_next(s, q, s_p, signed_check=True)
    t_new, ovf_t = calc_checked_next(t, q, t_p, signed_check=True)

    any_overflow = Or(ovf_r, ovf_s, ovf_t)

    return r_new, s_new, t_new, any_overflow

### Célula 7: `adicionar_regras`

Define as regras lógicas que governam o sistema:
1. **Inicialização**: Define os valores iniciais ($s=1, t=0$, etc.) quando $a, b > 0$.
2. **Transição**: Se o Invariante é válido, $r' \neq 0$ e **não há overflow**, o sistema avança para o próximo estado.
3. **Falha**: Se o sistema atinge um estado onde $r' \neq 0$ mas ocorre um erro matemático (overflow) ou uma condição proibida ($r=0$), a relação `Fail` torna-se verdadeira.

In [ ]:
def adicionar_regras(fp, Inv, Fail, vars_estado):

    a, b, r, r_p, s, s_p, t, t_p = vars_estado
    all_vars = [a, b, r, r_p, s, s_p, t, t_p]
    
    zero_bv = BitVecVal(0, N)

    fp.add_rule(ForAll([a, b],
                       Implies(
                           And(UGT(a, zero_bv), UGT(b, zero_bv)),
                           Inv(a, b,
                               a, b,                    
                               BitVecVal(1, N), BitVecVal(0, N), 
                               BitVecVal(0, N), BitVecVal(1, N))
                       )))

    r_new, s_new, t_new, any_overflow = calcular_transicao(r, r_p, s, s_p, t, t_p)

    fp.add_rule(ForAll(all_vars,
                       Implies(
                           And(
                               Inv(a, b, r, r_p, s, s_p, t, t_p),
                               r_p != zero_bv,
                               Not(any_overflow)
                           ),
                           Inv(a, b, r_p, r_new, s_p, s_new, t_p, t_new)
                       )))

    cond_error = Or(r == zero_bv, any_overflow)
    fp.add_rule(ForAll(all_vars,
                       Implies(
                           And(
                               Inv(a, b, r, r_p, s, s_p, t, t_p),
                               r_p != zero_bv,
                               cond_error
                           ),
                           Fail()
                       )))

### Célula 8: `executar_verificacao`

Realiza a consulta (`query`) ao solver para verificar se é possível atingir a relação `Fail`.
* **unsat**: O sistema é **Seguro**. Não existe caminho para o erro.
* **sat**: O sistema é **Inseguro**. O solver encontrou um caminho (contra-exemplo) que leva ao erro.

In [ ]:
def executar_verificacao(fp, Fail):
    res = fp.query(Fail)

    if res == unsat:
        print("\n RESULTADO: SEGURO (unsat)")
        print("  O Fixedpoint provou que não existe trace que leve a Fail (r=0 ou overflow).")
        print("\nInvariante calculado pelo solver:")
        try:
            print(fp.get_answer())
        except Z3Exception:
            print("  (fp.get_answer() não disponível)")
    elif res == sat:
        print("\n RESULTADO: INSEGURO (sat)")
        print("  O Fixedpoint encontrou um contra-exemplo que leva a Fail (r=0 ou overflow).")
        try:
            print("\nContra-exemplo / prova fornecida pelo solver:")
            print(fp.get_answer())
        except Z3Exception:
            pass
    else:
        print("\n RESULTADO: Unknown / Timeout")

### Célula 9: `verificar_exa_sfots_modular`

É a função principal que configura o solver, cria variáveis, regista relações, adiciona as regras e executa a verificação final.

In [ ]:
def verificar_exa_sfots_modular():
    print("  VERIFICAÇÃO EXA: SFOTS COM BITVECTORS (16-bit)")
    
    fp = configurar_solver()
    vars_estado = criar_variaveis()
    Inv, Fail = registar_relacoes(fp)
    adicionar_regras(fp, Inv, Fail, vars_estado)
    executar_verificacao(fp, Fail)

if __name__ == "__main__":
    verificar_exa_sfots_modular()